In [1]:
"""
安裝所需套件

"""
!pip install psycopg2
!pip install peewee

  Created wheel for peewee: filename=peewee-3.13.3-py3-none-any.whl size=127269 sha256=45ec59c23d652440a6ac491a24d716c56f93fd8db0c3dd75247489ae22103028
  Stored in directory: c:\users\big data\appdata\local\pip\cache\wheels\fa\c2\f9\3e8ca7e14368b03928bab7b5909c415414b6594ca051a64835
Successfully built peewee


In [ ]:
"""
Model.create(), for executing INSERT queries.
Model.save() and Model.update(), for executing UPDATE queries.
Model.delete_instance() and Model.delete(), for executing DELETE queries.
Model.select(), for executing SELECT queries.

"""

In [2]:
"""
引用套件
連接資料庫

"""
import peewee

# # SQLite database using WAL journal mode and 64MB cache.
# sqlite_db = peewee.SqliteDatabase('/path/to/app.db', pragmas={
#     'journal_mode': 'wal',
#     'cache_size': -1024 * 64})

# # Connect to a MySQL database on network.
# mysql_db = peewee.MySQLDatabase('my_app', user='app', password='db_password',
#                          host='10.1.0.8', port=3316)

# # Connect to a Postgres database.
# pg_db = peewee.PostgresqlDatabase('my_app', user='postgres', password='secret',
#                            host='10.1.0.9', port=5432)


db = peewee.PostgresqlDatabase('d17al439m29602', 
                        user='nnryazusdyyyfk', 
                        password='7c62d256a40b32a55fa525dc3ee243d277982ebc3a1b4a67b7b42be038ffce06',
                        host='ec2-52-202-146-43.compute-1.amazonaws.com', 
                        port=5432)



In [3]:
"""
立即連接資料庫，可以在此處就發現連線問題
若沒有特別使用 db.connect()，則第一次query 時才真正建立連線

"""
db.connect()

True

In [4]:
"""
定義資料表
    建立BaseModel類別，對應到資料庫中的表格。
    類別中的屬性對應到表格中的欄位。

在該類下創建子類別Meta
    Meta是一個內部類，
    用於定義peewee的Model類別的行為特性，
    指定dabatase 為 前面定義的db。

"""

# 定義Person資料表
class Person(peewee.Model):
    # 定義欄位
    name = peewee.CharField()
    birthday = peewee.DateField()
    
    # 指定使用的資料庫
    class Meta:
        database = db

# 定義Pet資料表
class Pet(peewee.Model):
    # 定義欄位
    owner = peewee.ForeignKeyField(Person, backref='pets')
    name = peewee.CharField()
    animal_type = peewee.CharField()

    # 指定使用的資料庫
    class Meta:
        database = db

In [5]:
"""
執行create_table()方法，將表格實際創建出來

"""

db.create_tables([Person, Pet])

In [7]:
"""
insert

    Storing data
    Let’s begin by populating the database with some people. 
    We will use the save() and create() methods to add and update people's records.

"""

# from datetime import date

# # 第一種插入資料方式：建立instance時直接提供參數，呼叫save()儲存資料。
# uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15))
# uncle_bob.save() # bob is now stored in the database # Returns: 1

# # 第二種插入資料方式：呼叫create()直接儲存資料。
# grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1))
# herb = Person.create(name='Herb', birthday=date(1950, 5, 5))

# bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
# herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
# herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
# herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')

"\ninsert\n\n    Storing data\n    Let’s begin by populating the database with some people. \n    We will use the save() and create() methods to add and update people's records.\n\n"

In [6]:
"""
第一種插入資料方式：
    建立instance時直接提供參數
    隨後呼叫save()儲存資料。

"""

from datetime import date

uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15))
uncle_bob.save() # bob is now stored in the database # Returns: 1


1

In [7]:
"""
第二種插入資料方式：
    呼叫create()直接儲存資料。

"""

from datetime import date

grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1))
herb = Person.create(name='Herb', birthday=date(1950, 5, 5))


In [8]:
"""
請練習在Pet資料表中也插入資料

"""

bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')


In [9]:
"""
select

Single record

"""

# To get a single record from the database, use Select.get():
grandma = Person.select().where(Person.name == 'Herb').get()
print(grandma.name, grandma.birthday)

# We can also use the equivalent shorthand Model.get():
grandma2 = Person.get(Person.name == 'Grandma')
print(grandma2.name, grandma2.birthday)


Herb 1950-05-05
Grandma 1935-03-01


In [10]:
"""
select

lists of records

"""
print(type(Person.select()))  # 可用迴圈遍歷，但不是真的python的list

# 使用select()，選取全部資料，再用迴圈一一列出。
for person in Person.select():
    print(person.id, person.name, person.birthday)

print("---分隔線---")

# 使用更多SQL語句
query = Pet.select().where(Pet.animal_type == 'cat')

for pet in query:
    print("Pet's name: ", pet.name, "; ", "Owner's name: ", pet.owner.name)

<class 'peewee.ModelSelect'>
1 Bob 1960-01-15
2 Grandma 1935-03-01
3 Herb 1950-05-05
---分隔線---
Pet's name:  Kitty ;  Owner's name:  Bob
Pet's name:  Mittens ;  Owner's name:  Herb
Pet's name:  Mittens Jr ;  Owner's name:  Herb


In [11]:
"""
update

    When you call save(), the number of rows modified is returned.

"""
from datetime import date

# step1 選出要變更的資料
grandmaUpdate = Person.select().where(Person.name == 'Grandma').get()
print(type(grandmaUpdate))

# step2 變更欄位
grandmaUpdate.name = 'Grandma L.'

# step3 儲存變更
grandmaUpdate.save()  # Update grandma's name in the database.
# Returns: 1

<Model: Person>


1

In [12]:
"""
delete

"""
# 刪除單筆資料：Model.delete_instance()
person = Person.get(Person.name == 'Grandma L.')
person.delete_instance()  # Returns the number of rows deleted.
# 1


# # 刪除多筆資料：Model.delete()
# query = Person.delete().where(Person.id > 1)
# query.execute()  # Returns the number of rows deleted.

1

In [13]:
"""
關閉資料庫連線

"""

if db:
    db.close()